## Object detection, object cropping and object counting

In [1]:
import os
import cv2
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors

# Load the model
model = YOLO("./models/yolov8x.pt")
names = model.names

results = None

# Load the image
image_path = "img2.jpg"

im0 = cv2.imread(image_path)
# Check if the image was successfully loaded
assert im0 is not None, "Error reading image file"

# Create a base directory for cropped images
crop_base_dir = "cropped_objects"
if not os.path.exists(crop_base_dir):
    os.mkdir(crop_base_dir)

# Predict objects in the image
results = model.predict(im0, show=False, device=0)
# Extract bounding boxes and class IDs from the results
boxes = results[0].boxes.xyxy.cpu().tolist()
clss = results[0].boxes.cls.cpu().tolist()

# Initialize a counter for cropped images
idx = 0
# Dictionary to keep count of objects for each class
class_counts = {}

# Check if there are any detected objects
if boxes is not None:
    for box, cls in zip(boxes, clss):
        idx += 1

        # Crop the detected object from the image
        crop_obj = im0[int(box[1]):int(box[3]), int(box[0]):int(box[2])]

        # Get the class name for the detected object
        class_name = names[int(cls)]
        # Create a directory for the class if it doesn't exist
        class_dir = os.path.join(crop_base_dir, names[int(cls)])
        if not os.path.exists(class_dir):
            os.mkdir(class_dir)

        # Save the cropped object in the class directory
        cv2.imwrite(f"{image_path.split('.')[0]}_object_cropping_output.jpg", im0)
        cv2.imwrite(os.path.join(class_dir, f"{image_path.split('.')[0]}_{idx}.png"), crop_obj)

        # Update the class count
        if class_name in class_counts:
            class_counts[class_name] += 1
        else:
            class_counts[class_name] = 1

# Create an annotator for visualizing the results
annotator = Annotator(im0, line_width=2, example=names)
# Check if there are any detected objects
if boxes is not None:
    for box, cls in zip(boxes, clss):
        # Annotate the bounding box on the image
        annotator.box_label(box, color=colors(int(cls), True), label=names[int(cls)])

# # Display the annotated image
# cv2.imshow("ultralytics", im0)

# Save the annotated image
cv2.imwrite(f"{image_path.split('.')[0]}_object_cropping_output.jpg", im0)

# Wait for a key press and then close all OpenCV windows
cv2.waitKey(0)
cv2.destroyAllWindows()

# Print the counts of objects per class
for class_name, count in class_counts.items():
    print(f"Class '{class_name}' has {count} objects.")


0: 480x640 5 persons, 1 bottle, 1 cup, 3 chairs, 2 potted plants, 1 vase, 135.0ms
Speed: 7.1ms preprocess, 135.0ms inference, 73.5ms postprocess per image at shape (1, 3, 480, 640)
Class 'person' has 5 objects.
Class 'vase' has 1 objects.
Class 'potted plant' has 2 objects.
Class 'chair' has 3 objects.
Class 'bottle' has 1 objects.
Class 'cup' has 1 objects.


## Face detection and facial emotion recognition

### Testing

In [2]:
from deepface import DeepFace
from collections import Counter

In [3]:
backends = [
  'opencv', 
  'ssd', 
  'dlib', 
  'mtcnn', 
  'fastmtcnn',
  'retinaface', 
  'mediapipe',
  'yolov8',
  'yunet',
  'centerface',
]

alignment_modes = [True, False]

In [4]:
demographies = DeepFace.analyze(
    img_path='cropped_objects\person\img2_5.png',
    detector_backend=backends[7],
    align=alignment_modes[0],
    actions=['emotion'],
)

demographies

[{'emotion': {'angry': 7.590587602912535e-08,
   'disgust': 1.5783188422663005e-15,
   'fear': 8.595904432695534e-07,
   'happy': 99.9997615814209,
   'sad': 3.6539052672424077e-06,
   'surprise': 0.00012953510122315492,
   'neutral': 0.00010266554681948037},
  'dominant_emotion': 'happy',
  'region': {'x': 230,
   'y': 43,
   'w': 120,
   'h': 180,
   'left_eye': (340, 117),
   'right_eye': (295, 117)},
  'face_confidence': 0.79}]

In [5]:
# Folder path for person class
person_class_folder = os.path.join(crop_base_dir, 'person')

# List to store dominant emotions
dominant_emotions = []

# Check if the person class folder exists
if os.path.exists(person_class_folder):
    # Iterate through all images in the person class folder
    for image_name in os.listdir(person_class_folder):
        image_path = os.path.join(person_class_folder, image_name)
        
        # Perform face detection and emotion recognition
        try:
            demographies = DeepFace.analyze(
                img_path=image_path,
                detector_backend=backends[7],  # RetinaFace backend
                align=alignment_modes[0],  # Alignment enabled
                actions=['emotion'],  # Only analyze emotions
            )
            # Get the dominant emotion for each image and add it to the list
            dominant_emotion = demographies[0]['dominant_emotion']
            dominant_emotions.append(dominant_emotion)
            # # Print the results for each image
            # print(f"Results for {image_name}: {demographies}")
        except Exception as e:
            continue

# Count the occurrences of each dominant emotion
emotion_counts = Counter(dominant_emotions)
# Find the most common dominant emotion
most_common_emotion = emotion_counts.most_common(1)[0][0]

# Print the most common dominant emotion
print(f"The most common dominant emotion is '{most_common_emotion}'.")

The most common dominant emotion is 'happy'.


### Final code 1 (include visulization for each image)

In [6]:
from deepface import DeepFace
from collections import Counter

# Folder path for person class
person_class_folder = os.path.join(crop_base_dir, 'person')

# Folder path for visualization of face detection and facial emotion recognition
emotion_visualization = "./emotion_visualization"

# Create a directory for the class if it doesn't exist
if not os.path.exists(emotion_visualization):
    os.mkdir(emotion_visualization)

# List to store dominant emotions
dominant_emotions = []

# Check if the person class folder exists
if os.path.exists(person_class_folder):
    # Iterate through all images in the person class folder
    for image_name in os.listdir(person_class_folder):
        image_path = os.path.join(person_class_folder, image_name)
        
        # Perform face detection and emotion recognition
        try:
            demographies = DeepFace.analyze(
                img_path=image_path,
                detector_backend='yolov8',  # RetinaFace backend
                align=True,  # Alignment enabled
                actions=['emotion'],  # Only analyze emotions
            )
            # Get the dominant emotion for each image and add it to the list
            dominant_emotion = demographies[0]['dominant_emotion']
            dominant_emotions.append(dominant_emotion)

            # Visualize the face detection and emotion on the image
            face_box = demographies[0]['region']
            x, y, w, h = face_box['x'], face_box['y'], face_box['w'], face_box['h']
            image = cv2.imread(image_path)
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(image, dominant_emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            # Save the visualized image
            output_image_path = os.path.join(emotion_visualization, f"visualized_{image_name}")
            cv2.imwrite(output_image_path, image)

            # # Print the results for each image
            # print(f"Results for {image_name}: {demographies}")
        except Exception as e:
            continue

# Count the occurrences of each dominant emotion
emotion_counts = Counter(dominant_emotions)
# Find the most common dominant emotion
most_common_emotion = emotion_counts.most_common(1)[0][0]

# Print the most common dominant emotion
print(f"The most common dominant emotion is '{most_common_emotion}'.")

The most common dominant emotion is 'happy'.


## Final code 2 (without visualization for each image)

In [7]:
from deepface import DeepFace
from collections import Counter

# Folder path for person class
person_class_folder = os.path.join(crop_base_dir, 'person')

# List to store dominant emotions
dominant_emotions = []

# Check if the person class folder exists
if os.path.exists(person_class_folder):
    # Iterate through all images in the person class folder
    for image_name in os.listdir(person_class_folder):
        image_path = os.path.join(person_class_folder, image_name)
        
        # Perform face detection and emotion recognition
        try:
            demographies = DeepFace.analyze(
                img_path=image_path,
                detector_backend='yolov8',  # RetinaFace backend
                align=True,  # Alignment enabled
                actions=['emotion'],  # Only analyze emotions
            )
            # Get the dominant emotion for each image and add it to the list
            dominant_emotion = demographies[0]['dominant_emotion']
            dominant_emotions.append(dominant_emotion)

            # # Print the results for each image
            # print(f"Results for {image_name}: {demographies}")
        except Exception as e:
            continue

# Count the occurrences of each dominant emotion
emotion_counts = Counter(dominant_emotions)
# Find the most common dominant emotion
most_common_emotion = emotion_counts.most_common(1)[0][0]

# Print the most common dominant emotion
print(f"The most common dominant emotion is '{most_common_emotion}'.")

The most common dominant emotion is 'happy'.
